In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
import time
# Function to calculate CAGR
def calculate_cagr(start_value, end_value, periods):
    if start_value <= 0 or end_value <= 0 or periods <= 0:
        return np.nan
    cagr = ((end_value / start_value) ** (1 / periods) - 1) * 100
    return cagr

# Main function to fetch and compute metrics
def get_peaceful_investing_data_fmp(ticker, api_key, years=10):
    base_url = "https://financialmodelingprep.com/api/v3"
    
    # Fetch data with error handling
    income_response = requests.get(f"{base_url}/income-statement/{ticker}?limit={years}&apikey={api_key}")
    if income_response.status_code != 200:
        return f"Error fetching income data: {income_response.text}"
    income = income_response.json()
    
    balance_response = requests.get(f"{base_url}/balance-sheet-statement/{ticker}?limit={years}&apikey={api_key}")
    if balance_response.status_code != 200:
        return f"Error fetching balance data: {balance_response.text}"
    balance = balance_response.json()
    
    cashflow_response = requests.get(f"{base_url}/cash-flow-statement/{ticker}?limit={years}&apikey={api_key}")
    if cashflow_response.status_code != 200:
        return f"Error fetching cashflow data: {cashflow_response.text}"
    cashflow = cashflow_response.json()
    
    key_metrics_response = requests.get(f"{base_url}/key-metrics/{ticker}?limit={years}&apikey={api_key}")
    if key_metrics_response.status_code != 200:
        return f"Error fetching key metrics: {key_metrics_response.text}"
    key_metrics = key_metrics_response.json()
    
    profile_response = requests.get(f"{base_url}/profile/{ticker}?apikey={api_key}")
    if profile_response.status_code != 200:
        return f"Error fetching profile: {profile_response.text}"
    profile_data = profile_response.json()
    if not profile_data:
        return "No profile data available for this ticker"
    profile = profile_data[0]
    
    quote_response = requests.get(f"{base_url}/quote/{ticker}?apikey={api_key}")
    if quote_response.status_code != 200:
        return f"Error fetching quote: {quote_response.text}"
    quote_data = quote_response.json()
    if not quote_data:
        return "No quote data available for this ticker"
    quote = quote_data[0]
    
    if not income or not balance or not cashflow:
        return "Incomplete financial data fetched"
    
    return income, balance, cashflow, key_metrics, profile, quote

def compute_metrics_fmp(income, balance, cashflow, key_metrics, profile, quote):

    # Convert to DataFrames (reverse to chronological: oldest first)
    income_df = pd.DataFrame(income[::-1])
    balance_df = pd.DataFrame(balance[::-1])
    cashflow_df = pd.DataFrame(cashflow[::-1])
    key_metrics_df = pd.DataFrame(key_metrics[::-1])
    
    # Current year index (latest)
    latest = -1
    
    # Compute metrics with defaults for missing data
    metrics = {}
    metrics['ticker'] = income_df['symbol'][0]
    # Return on Equity (ROE) - Latest and 3-year average
    net_income = income_df['netIncome']
    equity = balance_df['totalStockholdersEquity'].shift(1)
    metrics['ROE'] = (net_income.iloc[latest] / equity.iloc[latest] * 100) if equity.iloc[latest] != 0 else 0
    roe_3yr = [(net_income.iloc[i] / equity.iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if equity.iloc[i] != 0]
    metrics['ROE (3yr avg)'] = np.mean(roe_3yr) if roe_3yr else 0
    
    # Return on Capital Employed (ROCE) - Latest and 3-year average
    ebit = income_df['operatingIncome']
    capital_employed = balance_df['totalAssets'] - balance_df['totalCurrentLiabilities']
    metrics['ROCE'] = (ebit.iloc[latest] / capital_employed.iloc[latest] * 100) if capital_employed.iloc[latest] != 0 else 0
    roce_3yr = [(ebit.iloc[i] / capital_employed.iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if capital_employed.iloc[i] != 0]
    metrics['ROCE (3yr avg)'] = np.mean(roce_3yr) if roce_3yr else 0
    
    # Net Fixed Assets (NFA) for each year
    net_fixed_assets = balance_df['propertyPlantEquipmentNet']
    
    # Net Fixed Asset Turnover (NFAT) - Calculate for each year, then 3-year average
    revenue = income_df['revenue']
    nfat_values = []
    for i in range(1, len(income_df)):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
        nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    if len(income_df) == 1:
        avg_nfa_year = net_fixed_assets.iloc[latest]
        nfat_year = revenue.iloc[latest] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    metrics['NFAT'] = nfat_values[-1] if nfat_values else 0  # Latest year's NFAT
    nfat_3yr = nfat_values[-min(3, len(nfat_values)):]
    metrics['NFAT (3yr avg)'] = np.mean(nfat_3yr) if nfat_3yr else metrics['NFAT']  # Fallback to latest if <3
    
    # Net Profit Margin (NPM) - 3-year average
    npm_3yr = [(income_df['netIncome'].iloc[i] / income_df['revenue'].iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if income_df['revenue'].iloc[i] != 0]
    metrics['NPM'] = np.mean(npm_3yr) if npm_3yr else 0
    
    # Dividend Payout Ratio (DPR) - 3-year average
    dividends_paid = cashflow_df['dividendsPaid'].abs()
    dpr_3yr = [(dividends_paid.iloc[i] / income_df['netIncome'].iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if income_df['netIncome'].iloc[i] != 0]
    metrics['DPR'] = np.mean(dpr_3yr) if dpr_3yr else 0
    
    # Retention Ratio (1 - DPR)
    metrics['Retention Ratio'] = (1 - (metrics['DPR'] / 100)) * 100
    
    # Depreciation Rate (Dep as % of NFA) - 3-year average
    dep = cashflow_df['depreciationAndAmortization']
    dep_3yr = []
    for i in range(-min(3, len(income_df)), 0):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.shift(1).iloc[i]) / 2 if len(balance_df) > 1 and i > -len(income_df) + 1 else net_fixed_assets.iloc[i]
        dep_year = (dep.iloc[i] / avg_nfa_year * 100) if avg_nfa_year != 0 else 0
        dep_3yr.append(dep_year)
    metrics['Dep'] = np.mean(dep_3yr) if dep_3yr else 0
    
    # Self Sustainable Growth Rate (SSGR) = NFAT * NPM * (1 - DPR) - Dep
    npm_decimal = metrics['NPM'] / 100
    dpr_decimal = metrics['DPR'] / 100
    dep_decimal = metrics['Dep'] / 100
    metrics['SSGR'] = (metrics['NFAT (3yr avg)'] * npm_decimal * (1 - dpr_decimal) - dep_decimal) * 100  # As %
    
    # Avg NPM (over 3 years) - Already handled above
    metrics['Av NPM (over 3 years)'] = metrics['NPM']
    
    # Avg NFAT (over 3 years) - Already handled above
    metrics['Au NFA/T (over 3 years)'] = metrics['NFAT (3yr avg)']
    
    # Avg Dep % NFA (over 3 years) - Already handled above
    metrics['Av Dep%NFA (over 3 years)'] = metrics['Dep']
    
    # Avg Retention Ratio (over 3 years)
    ret_3yr = [(1 - (dividends_paid.iloc[i] / income_df['netIncome'].iloc[i])) * 100 for i in range(-min(3, len(income_df)), 0) if income_df['netIncome'].iloc[i] != 0]
    metrics['Au Retention ratio (over 3 years)'] = np.mean(ret_3yr) if ret_3yr else 0
    
    # Debt to Equity (d/e)
    metrics['d/e'] = key_metrics_df['debtToEquity'].iloc[latest] if 'debtToEquity' in key_metrics_df.columns and not key_metrics_df.empty else (balance_df['totalDebt'].iloc[latest] / balance_df['totalStockholdersEquity'].iloc[latest] if balance_df['totalStockholdersEquity'].iloc[latest] != 0 else 0)
    
    # Interest Coverage
    interest_exp = abs(income_df['interestExpense'].iloc[latest])  # Make positive
    metrics['Interest coverage'] = (income_df['operatingIncome'].iloc[latest] / interest_exp) if interest_exp != 0 else float('inf')
    
    # Tax %
    metrics['tax %'] = (income_df['incomeTaxExpense'].iloc[latest] / income_df['incomeBeforeTax'].iloc[latest] * 100) if 'incomeBeforeTax' in income_df.columns and income_df['incomeBeforeTax'].iloc[latest] != 0 else 0
    
    # Cumulative PAT (cPAT) - Sum over last 5 years
    metrics['cPAT'] = income_df['netIncome'][-5:].sum() if len(income_df) >= 5 else income_df['netIncome'].sum()
    
    # CFO (latest)
    metrics['CFO'] = cashflow_df['netCashProvidedByOperatingActivities'].iloc[latest]
    
    # Cumulative CFO (cCFO)
    metrics['cCFO'] = cashflow_df['netCashProvidedByOperatingActivities'][-5:].sum() if len(cashflow_df) >= 5 else cashflow_df['netCashProvidedByOperatingActivities'].sum()
    
    # Cumulative CFO / cPAT
    metrics['cCFO/cPAT'] = (metrics['cCFO'] / metrics['cPAT']) if metrics['cPAT'] != 0 else 0
    
    # ROA (p/a)
    metrics['p/a'] = (income_df['netIncome'].iloc[latest] / balance_df['totalAssets'].shift(1).iloc[latest] * 100) if balance_df['totalAssets'].shift(1).iloc[latest] != 0 else 0
    
    # Price to Earnings (p/e)
    pe = quote.get('pe', float('inf'))
    metrics['p/e'] = pe
    
    # Earnings Yield (EY)
    metrics['EY'] = (income_df['eps'].iloc[latest] / quote['price'] * 100) if quote['price'] != 0 else 0
    
    # Earnings Growth 5yr CAGR
    eps_values = income_df['eps'][-6:] if len(income_df) >= 6 else income_df['eps']
    periods = len(eps_values) - 1
    metrics['Earnings Growth 5yr cagr'] = calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods) if periods > 0 else 0
    
    # PEG
    metrics['PEG'] = (pe / metrics['Earnings Growth 5yr cagr']) if metrics['Earnings Growth 5yr cagr'] != 0 else float('inf')
    
    # No. shares (cr) - in crores
    shares_out = profile.get('sharesOutstanding', quote['marketCap'] / quote['price'] if quote['price'] != 0 else 0)
    metrics['no. shares (cr)'] = shares_out / 1e7
    
    # Price to Sales (p/s)
    metrics['p/s'] = quote.get('priceToSalesRatio', 0)
    
    # NFA + CWIP
    cwip = balance_df.get('constructionInProgress', pd.Series([0]*len(balance_df))).iloc[latest]
    metrics['NFA + CWIP'] = net_fixed_assets.iloc[latest] + cwip
    
    # Capex = (NFA + CWIP end) - (NFA + CWIP start) + Dep
    if len(balance_df) >= 2:
        nfa_cwip_end = net_fixed_assets.iloc[latest] + cwip
        nfa_cwip_start = net_fixed_assets.iloc[latest-1] + balance_df.get('constructionInProgress', pd.Series([0]*len(balance_df))).iloc[latest-1]
        metrics['Capex'] = nfa_cwip_end - nfa_cwip_start + dep.iloc[latest]
    else:
        metrics['Capex'] = abs(cashflow_df['capitalExpenditure'].iloc[latest])  # Fallback
    
    # Free Cash Flow (FCF) = CFO - Capex
    metrics['FCF'] = metrics['CFO'] - metrics['Capex']
    
    # FCF%
    metrics['FCF%'] = (metrics['FCF'] / net_income.iloc[latest] * 100) if net_income.iloc[latest] != 0 else 0
    
    # Dividend Yield (DV)
    per_share_div = (dividends_paid.iloc[latest] / shares_out) if shares_out != 0 else 0
    metrics['DV'] = (per_share_div / quote['price'] * 100) if quote['price'] != 0 else 0
    
    # Mcap (cr)
    metrics['Mcap (cr)'] = quote['marketCap'] / 1e7
    
    # d/e decreasing trend 5 yrs
    de_ratios = [balance_df['totalDebt'].iloc[i] / balance_df['totalStockholdersEquity'].iloc[i] if balance_df['totalStockholdersEquity'].iloc[i] != 0 else 0 for i in range(-5, 0) if len(balance_df) >= 5]
    metrics['d/e decreasing trend 5 yrs'] = all(de_ratios[i] > de_ratios[i+1] for i in range(len(de_ratios)-1)) if len(de_ratios) > 1 else False
    
    # Financial Analysis Criteria
    sales_values = income_df['revenue'][-6:] if len(income_df) >= 6 else income_df['revenue']
    sales_periods = len(sales_values) - 1
    sales_cagr = calculate_cagr(sales_values.iloc[0], sales_values.iloc[-1], sales_periods) if sales_periods > 0 else 0
    metrics['Sales cagr >15%'] = sales_cagr > 15
    metrics['npm >8%'] = metrics['NPM'] > 8
    metrics['Tax payout >25%'] = metrics['tax %'] > 25
    metrics['Interest coverage >3'] = metrics['Interest coverage'] > 3
    metrics['d/e <0.5'] = metrics['d/e'] < 0.5
    metrics['CFO >0'] = metrics['CFO'] > 0
    metrics['net cash flow positive'] = cashflow_df['netChangeInCash'].iloc[latest] > 0
    metrics['cCFO > PAT'] = metrics['cCFO/cPAT'] > 1
    
    # Valuation Analysis
    metrics['p/e <10'] = pe < 10
    metrics['peg <1'] = metrics['PEG'] < 1
    metrics['EY >7%'] = metrics['EY'] > 7
    metrics['p/b <3'] = quote.get('priceToBookRatio', float('inf')) < 3
    metrics['DV >3%'] = metrics['DV'] > 3
    
    # Margin of Safety
    metrics['EY >7'] = metrics['EY'] > 7  # Duplicate
    metrics['sgr > Sales growth (very linear)'] = metrics['SSGR'] > sales_cagr
    metrics['FCF/CFO'] = (metrics['FCF'] / metrics['CFO']) if metrics['CFO'] != 0 else 0
    
    # Current Price
    metrics['Current Price'] = quote['price']

    # Raw Financial Data
    # Raw Financial Data (corrected columns)
    metrics['Market Cap'] = quote.get('marketCap', 0)  # In original currency
    metrics['Net Income'] = net_income.iloc[latest] if not pd.isna(net_income.iloc[latest]) else 0
    metrics['Total Revenue'] = revenue.iloc[latest] if not pd.isna(revenue.iloc[latest]) else 0
    metrics['Total Assets'] = balance_df['totalAssets'].iloc[latest] if not pd.isna(balance_df['totalAssets'].iloc[latest]) else 0
    metrics['Total Liabilities'] = balance_df['totalLiabilities'].iloc[latest] if not pd.isna(balance_df['totalLiabilities'].iloc[latest]) else 0
    metrics['Total Stockholders Equity'] = equity.iloc[latest] if not pd.isna(equity.iloc[latest]) else 0
    metrics['Total Debt'] = balance_df['totalDebt'].iloc[latest] if not pd.isna(balance_df['totalDebt'].iloc[latest]) else 0
    metrics['Cash and Cash Equivalents'] = balance_df['cashAndCashEquivalents'].iloc[latest] if not pd.isna(balance_df['cashAndCashEquivalents'].iloc[latest]) else 0
    metrics['Current Assets'] = balance_df['totalCurrentAssets'].iloc[latest] if not pd.isna(balance_df['totalCurrentAssets'].iloc[latest]) else 0
    metrics['Current Liabilities'] = balance_df['totalCurrentLiabilities'].iloc[latest] if not pd.isna(balance_df['totalCurrentLiabilities'].iloc[latest]) else 0
    metrics['Operating Cash Flow'] = metrics['CFO']  # Already calculated, stored raw
    metrics['Capital Expenditure'] = abs(cashflow_df['capitalExpenditure'].iloc[latest]) if not pd.isna(cashflow_df['capitalExpenditure'].iloc[latest]) else 0
    metrics['Dividends Paid'] = dividends_paid.iloc[latest] if not pd.isna(dividends_paid.iloc[latest]) else 0
    metrics['Depreciation & Amortization'] = dep.iloc[latest] if not pd.isna(dep.iloc[latest]) else 0
    metrics['Interest Expense'] = interest_exp if not pd.isna(interest_exp) else 0
    metrics['Income Tax Expense'] = income_df['incomeTaxExpense'].iloc[latest] if not pd.isna(income_df['incomeTaxExpense'].iloc[latest]) else 0
    metrics['Shares Outstanding'] = shares_out
    metrics['Current Price'] = quote['price']  # Already calculated, stored raw
    metrics['Net Fixed Assets'] = net_fixed_assets.iloc[latest] if not pd.isna(net_fixed_assets.iloc[latest]) else 0
    metrics['Construction in Progress'] = cwip if not pd.isna(cwip) else 0
    metrics['Net Debt'] = metrics['Total Debt'] - metrics['Cash and Cash Equivalents'] if not pd.isna(metrics['Total Debt']) and not pd.isna(metrics['Cash and Cash Equivalents']) else 0



    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])

    #metrics_df.to_csv('fmp/aapl.csv')
    metrics_df.to_csv(f'fmp/{metrics['ticker']}.csv')
    
    return metrics


def retrieve_data_from_yfinance(ticker):
    stock = yf.Ticker(ticker)

    return stock
    
def null_check(c):
    if c!=np.nan and c!=None and pd.notnull(c):
        return True
    else:
        return False


def get_means(num_list, min_n=2):
    #min_n: how many entries is minimum
    num_list = [c for c in num_list if null_check(c)]
    mean_cal =  np.mean(num_list)

    if null_check(mean_cal) and len(num_list)>=min_n:
        return mean_cal
    else:
        return -1

# Main function to fetch and compute metrics
def get_peaceful_investing_metrics_from_yfinance(stock, years=5):  # Default to 5 years as yfinance often provides ~4-5
    #stock = yf.Ticker(ticker)
    # Fetch data
    income = stock.financials.transpose()  # Transpose to rows=years, columns=items; recent first
    balance = stock.balance_sheet.transpose()
    cashflow = stock.cashflow.transpose()
    info = stock.info  # Dict for quote/profile/metrics
    
    # Earnings history for EPS/Revenue (annual)
    #earnings = stock.earnings  # pd.DataFrame with Revenue, Earnings
    
    if income.empty or balance.empty or cashflow.empty:
        return "Error fetching data for this ticker"
    
    # Reset index to make date a column; sort oldest first for calculations

    

    income = income.reset_index().rename(columns={'index': 'Date'})
    balance = balance.reset_index().rename(columns={'index': 'Date'})
    cashflow = cashflow.reset_index().rename(columns={'index': 'Date'})
    
    income['Date'] = pd.to_datetime(income['Date'])
    balance['Date'] = pd.to_datetime(balance['Date'])
    cashflow['Date'] = pd.to_datetime(cashflow['Date'])

    income = income.sort_values('Date')
    balance = balance.sort_values('Date')
    cashflow = cashflow.sort_values('Date')
    # Current year index (latest)
    latest = -1
    
    # Available years
    num_years = len(income)
    
    # Compute metrics with defaults for missing data
    metrics = {}
    metrics['ticker'] = stock.ticker
    metrics['longName'] =stock.info['longName']
    # Return on Equity (ROE) - Latest and 3-year average
    net_income = income.get('Net Income', pd.Series([0]*num_years))
    equity = balance.get('Stockholders Equity', pd.Series([0]*num_years)).shift(1)
    #metrics['ROE'] = (net_income.iloc[latest] / equity.iloc[latest] * 100) if equity.iloc[latest] != 0 else 0
    #roe_3yr = [(net_income.iloc[i] / equity.iloc[i] * 100) for i in range(-min(3, num_years), 0) if equity.iloc[i] != 0]
    #metrics['ROE (3yr avg)'] = np.mean(roe_3yr) if roe_3yr else 0
    
    # Return on Capital Employed (ROCE) - Latest and 3-year average
    ebit = income.get('EBIT', pd.Series([0]*num_years))
    capital_employed = balance.get('Total Assets', pd.Series([0]*num_years)) - balance.get('Current Liabilities', pd.Series([0]*num_years))
    metrics['ROCE'] = (ebit.iloc[latest] / capital_employed.iloc[latest] * 100) if capital_employed.iloc[latest] != 0 else 0
    roce_3yr = [(ebit.iloc[i] / capital_employed.iloc[i] * 100) for i in range(-min(3, num_years), 0) if capital_employed.iloc[i] != 0]
    metrics['ROCE (3yr avg)'] = get_means(roce_3yr)#np.mean(roce_3yr) if roce_3yr else 0
    
    # Net Fixed Assets (NFA) for each year
    net_fixed_assets = balance.get('Net PPE', pd.Series([0]*num_years))
    
    # Net Fixed Asset Turnover (NFAT) - Calculate for each year, then 3-year average
    revenue = income.get('Total Revenue', pd.Series([0]*num_years))
    nfat_values = []
    try:
        for i in range(1, num_years):
            avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
            nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
            nfat_values.append(nfat_year)
    except:
        for i in range(1, num_years-1):
            avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
            nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
            nfat_values.append(nfat_year)
    if num_years == 1:
        avg_nfa_year = net_fixed_assets.iloc[latest]
        nfat_year = revenue.iloc[latest] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    metrics['NFAT'] = nfat_values[-1] if nfat_values else 0  # Latest year's NFAT
    nfat_3yr = nfat_values[-min(3, len(nfat_values)):]

    #print(f"nfat_3yr : {nfat_3yr}")
    metrics['NFAT (3yr avg)'] = get_means(nfat_3yr)#np.mean(nfat_3yr) if nfat_3yr else metrics['NFAT']  # Fallback to latest if <3
    
    # Net Profit Margin (NPM) - 3-year average
    npm_3yr = [(net_income.iloc[i] / revenue.iloc[i] * 100) for i in range(-min(3, num_years), 0) if revenue.iloc[i] != 0]
    metrics['NPM'] = get_means(npm_3yr)#np.mean(npm_3yr) if npm_3yr else 0
    
    # Dividend Payout Ratio (DPR) - 3-year average
    dividends_paid = cashflow.get('Cash Dividends Paid', pd.Series([0]*num_years)).abs()
    dpr_3yr = [(dividends_paid.iloc[i] / net_income.iloc[i] * 100) for i in range(-min(3, num_years), 0) if net_income.iloc[i] != 0]
    metrics['DPR'] = get_means(dpr_3yr)#np.mean(dpr_3yr) if dpr_3yr else 0
    
    # Retention Ratio (1 - DPR)
    metrics['Retention Ratio'] = (1 - (metrics['DPR'] / 100)) * 100
    
    # Depreciation Rate (Dep as % of NFA) - 3-year average
    dep = cashflow.get('Depreciation And Amortization', pd.Series([0]*num_years))
    dep_3yr = []
    for i in range(-min(3, num_years), 0):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.shift(1).iloc[i]) / 2 if num_years > 1 and i > -num_years + 1 else net_fixed_assets.iloc[i]
        dep_year = min((abs(dep.iloc[i]) / avg_nfa_year * 100) if avg_nfa_year != 0 else 0, 100)  # Use abs for dep, cap at 100%
        #print(avg_nfa_year)
        #if dep_year > 100:
        #    print(f"Warning: Dep % for year {i} exceeded 100%; capped to 100%. Check data for {ticker}.")
        dep_3yr.append(dep_year)
    metrics['Dep'] = get_means(dep_3yr)#np.mean(dep_3yr) if dep_3yr else 0
    
    # Self Sustainable Growth Rate (SSGR) = NFAT * NPM * (1 - DPR) - Dep
    npm_decimal = metrics['NPM'] / 100
    dpr_decimal = metrics['DPR'] / 100
    dep_decimal = metrics['Dep'] / 100
    metrics['SSGR'] = (metrics['NFAT (3yr avg)'] * npm_decimal * (1 - dpr_decimal) - dep_decimal) * 100  # As %
    
    # Avg NPM (over 3 years) - Already handled above
    metrics['Av NPM (over 3 years)'] = metrics['NPM']
    
    # Avg NFAT (over 3 years) - Already handled above
    metrics['Au NFA/T (over 3 years)'] = metrics['NFAT (3yr avg)']
    
    # Avg Dep % NFA (over 3 years) - Already handled above
    metrics['Av Dep%NFA (over 3 years)'] = metrics['Dep']
    
    # Avg Retention Ratio (over 3 years)
    ret_3yr = [(1 - (dividends_paid.iloc[i] / net_income.iloc[i])) * 100 for i in range(-min(3, num_years), 0) if net_income.iloc[i] != 0]
    metrics['Au Retention ratio (over 3 years)'] = get_means(ret_3yr) #np.mean(ret_3yr) if ret_3yr else 0

    # Debt to Equity (d/e)
    total_debt = balance.get('Total Debt', pd.Series([0]*num_years)).iloc[latest]
    metrics['d/e'] = (total_debt / equity.iloc[latest]) if equity.iloc[latest] != 0 else 0
    
    # Interest Coverage
    interest_exp = abs(income.get('Interest Expense', pd.Series([0]*num_years)).iloc[latest])
    metrics['Interest coverage'] = (income.get('Operating Income', pd.Series([0]*num_years)).iloc[latest] / interest_exp) if interest_exp != 0 else float('inf')
    
    # Tax %
    tax_exp = income.get('Income Tax Expense', pd.Series([0]*num_years)).iloc[latest] if 'Income Tax Expense' in income.columns else income.get('Tax Provision', pd.Series([0]*num_years)).iloc[latest] if 'Tax Provision' in income.columns else 0
    pretax = income.get('Pretax Income', pd.Series([0]*num_years)).iloc[latest]
    metrics['tax %'] = (tax_exp / pretax * 100) if pretax != 0 else 0
    
    # Cumulative PAT (cPAT) - Sum over available years (up to 5)
    metrics['cPAT'] = net_income[-min(5, num_years):].sum()
    
    # CFO (latest)
    metrics['CFO'] = cashflow.get('Operating Cash Flow', pd.Series([0]*num_years)).iloc[latest]
    
    # Cumulative CFO (cCFO)
    metrics['cCFO'] = cashflow.get('Operating Cash Flow', pd.Series([0]*num_years))[-min(5, num_years):].sum()
    
    # Cumulative CFO / cPAT
    metrics['cCFO/cPAT'] = (metrics['cCFO'] / metrics['cPAT']) if metrics['cPAT'] != 0 else 0
    
    # ROA (p/a)
    total_assets_prior = balance.get('Total Assets', pd.Series([0]*num_years)).shift(1).iloc[latest]
    metrics['p/a'] = (net_income.iloc[latest] / total_assets_prior * 100) if total_assets_prior != 0 else 0
    
    # Price to Earnings (p/e)
    pe = info.get('trailingPE', float('inf'))
    metrics['p/e'] = pe
    
    # Earnings Yield (EY)
    metrics['EY'] = (net_income.iloc[latest] / (info.get('sharesOutstanding', 1) * info.get('regularMarketPrice', 0)) * 100) if info.get('regularMarketPrice', 0) != 0 else 0
    
    # Earnings Growth 5yr CAGR
    eps_values = income.get('Basic EPS', pd.Series([0]*num_years))[-min(5, num_years):]
    if pd.isnull(eps_values.iloc[0]):
        eps_values = income.get('Basic EPS', pd.Series([0]*num_years))[-min(4, num_years):]
    periods = len(eps_values) - 1

    #print(f"eps : {eps_values}")
    metrics['Earnings Growth 5yr cagr'] = calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods) if periods > 0 else 0
    

    # Optional: Sales Growth 5yr CAGR
    revenue_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(5, num_years):]
    if pd.isnull(revenue_values.iloc[0]):
        revenue_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(4, num_years):]
    
    periods_revenue = len(revenue_values) - 1
    #print('2')
    sales_cagr = calculate_cagr(revenue_values.iloc[0], revenue_values.iloc[-1], periods_revenue) if periods_revenue > 0 else 0
    metrics['Sales Growth 5yr cagr'] = sales_cagr
    # PEG
    metrics['PEG'] = (pe / metrics['Earnings Growth 5yr cagr']) if metrics['Earnings Growth 5yr cagr'] != 0 else float('inf')
    
    # No. shares (cr) - in crores
    shares_out = info.get('sharesOutstanding', 0)
    metrics['no. shares (cr)'] = shares_out / 1e7

    metrics['Current Price'] = info.get('regularMarketPrice', 0)

    metrics['market cap'] = metrics['Current Price'] * shares_out
    metrics['d/e_market'] = total_debt/metrics['market cap']
    
    # Price to Sales (p/s)
    metrics['p/s'] = info.get('priceToSalesTrailing12Months', 0)
    
    # NFA + CWIP
    cwip = balance.get('Construction In Progress', pd.Series([0]*num_years)).iloc[latest]
    metrics['NFA + CWIP'] = net_fixed_assets.iloc[latest] + cwip
    
    # Capex = (NFA + CWIP end) - (NFA + CWIP start) + Dep
    if num_years >= 2:
        nfa_cwip_end = net_fixed_assets.iloc[latest] + cwip
        nfa_cwip_start = net_fixed_assets.iloc[latest-1] + balance.get('Construction In Progress', pd.Series([0]*num_years)).iloc[latest-1]
        metrics['Capex'] = nfa_cwip_end - nfa_cwip_start + dep.iloc[latest]
    else:
        metrics['Capex'] = abs(cashflow.get('Capital Expenditure', pd.Series([0]*num_years)).iloc[latest])  # Fallback
    
    metrics['Capex_from_cashflow_statement'] = abs(cashflow.get('Capital Expenditure', pd.Series([0]*num_years)).iloc[latest])

    # Free Cash Flow (FCF) = CFO - Capex
    metrics['FCF'] = metrics['CFO'] - metrics['Capex']
    
    # FCF%
    metrics['FCF%'] = (metrics['FCF'] / net_income.iloc[latest] * 100) if net_income.iloc[latest] != 0 else 0

   # Free Cash Flow (FCF) = CFO - Capex
    metrics['FCF_capex_from_cashflow'] = metrics['CFO'] - metrics['Capex_from_cashflow_statement']
    
    # FCF%
    metrics['FCF%_capex_from_cashflow'] = (metrics['FCF_capex_from_cashflow'] / net_income.iloc[latest] * 100) if net_income.iloc[latest] != 0 else 0
   

    # Dividend Yield (DV)
    per_share_div = (dividends_paid.iloc[latest] / shares_out) if shares_out != 0 else 0
    metrics['DV'] = (per_share_div / info.get('regularMarketPrice', 0) * 100) if info.get('regularMarketPrice', 0) != 0 else 0
    
    # Mcap (cr)
    metrics['Mcap (cr)'] = info.get('marketCap', 0) / 1e7
    
    # d/e decreasing trend 5 yrs
    try:
        de_ratios = [balance.get('Total Debt', pd.Series([0]*num_years)).iloc[i] / balance.get('Stockholders Equity', pd.Series([1]*num_years)).iloc[i] for i in range(-min(5, num_years), 0)]
    except:
        de_ratios = [balance.get('Total Debt', pd.Series([0]*num_years)).iloc[i] / balance.get('Stockholders Equity', pd.Series([1]*num_years)).iloc[i] for i in range(-min(3, num_years), 0)]
    metrics['d/e decreasing trend 5 yrs'] = all(de_ratios[j] > de_ratios[j+1] for j in range(len(de_ratios)-1)) if len(de_ratios) > 1 else False
    
    # Financial Analysis Criteria
    sales_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(6, num_years):]
    sales_periods = len(sales_values) - 1
    #sales_cagr = calculate_cagr(sales_values.iloc[0], sales_values.iloc[-1], sales_periods) if sales_periods > 0 else 0
    metrics['Sales cagr >15%'] = sales_cagr > 15
    metrics['npm >8%'] = metrics['NPM'] > 8
    metrics['Tax payout >25%'] = metrics['tax %'] > 25
    metrics['Interest coverage >3'] = metrics['Interest coverage'] > 3
    metrics['d/e <0.5'] = metrics['d/e'] < 0.5
    metrics['CFO >0'] = metrics['CFO'] > 0
    
    #metrics['net cash flow positive'] = cashflow.get('Net Change in Cash', pd.Series([0]*num_years)).iloc[latest] > 0
    metrics['cCFO > PAT'] = metrics['cCFO/cPAT'] > 1
    
    # Valuation Analysis
    metrics['p/e <10'] = pe < 10
    metrics['peg <1'] = metrics['PEG'] < 1
    metrics['EY >7%'] = metrics['EY'] > 7
    metrics['p/b <3'] = info.get('priceToBook', float('inf')) < 3
    metrics['DV >3%'] = metrics['DV'] > 3
    
    # Margin of Safety
    metrics['EY >7'] = metrics['EY'] > 7  # Duplicate
    metrics['sgr > Sales growth (very linear)'] = metrics['SSGR'] > sales_cagr
    metrics['FCF/CFO'] = (metrics['FCF'] / metrics['CFO']) if metrics['CFO'] != 0 else 0
    
    # Current Price
    metrics['Current Price'] = info.get('regularMarketPrice', 0)
    # Raw Financial Data
    # Raw Financial Data (corrected and enhanced columns with column existence check)
    metrics['Market Cap'] = info.get('marketCap', 0)  # In original currency
    metrics['Net Income'] = net_income.iloc[latest] if 'Net Income' in income.columns else 0
    metrics['Total Revenue'] = revenue.iloc[latest] if 'Total Revenue' in income.columns else 0
    metrics['Total Assets'] = balance['Total Assets'].iloc[latest] if 'Total Assets' in balance.columns else 0  # Fallback to sum if needed
    if metrics['Total Assets'] == 0 and 'Total Non Current Assets' in balance.columns and 'Current Assets' in balance.columns:
        metrics['Total Assets'] = (balance['Total Non Current Assets'].iloc[latest] + 
                                balance['Current Assets'].iloc[latest])
    metrics['Total Liabilities'] = balance['Total Liabilities Net Minority Interest'].iloc[latest] if 'Total Liabilities Net Minority Interest' in balance.columns else 0  # Use provided field
    if metrics['Total Liabilities'] == 0 and 'Total Non Current Liabilities Net Minority Interest' in balance.columns and 'Current Liabilities' in balance.columns:
        metrics['Total Liabilities'] = (balance['Total Non Current Liabilities Net Minority Interest'].iloc[latest] + 
                                    balance['Current Liabilities'].iloc[latest])
    metrics['Total Stockholders Equity'] = equity.iloc[latest] if 'Stockholders Equity' in balance.columns else 0
    
    metrics['Total Debt'] = (balance['Total Debt'].iloc[latest] if 'Total Debt' in balance.columns else balance['Long Term Debt'].iloc[latest] + balance['Current Debt'].iloc[latest] if 'Long Term Debt' in balance.columns and 'Current Debt' in balance.columns else 
                        balance['Total Non Current Liabilities Net Minority Interest'].iloc[latest] if 'Total Non Current Liabilities Net Minority Interest' in balance.columns else 0)
    metrics['Cash and Cash Equivalents'] = balance['Cash'].iloc[latest] if 'Cash' in balance.columns else balance['Other Current Assets'].iloc[latest] if 'Other Current Assets' in balance.columns else 0  # Fallback to Other Current Assets
    metrics['Current Assets'] = balance['Current Assets'].iloc[latest] if 'Current Assets' in balance.columns else 0
    if metrics['Current Assets'] == 0 and all(col in balance.columns for col in ['Other Current Assets', 'Hedging Assets Current', 'Assets Held For Sale Current', 'Prepaid Assets']):
        metrics['Current Assets'] = (balance['Other Current Assets'].iloc[latest] + 
                                    balance['Hedging Assets Current'].iloc[latest] + 
                                    balance['Assets Held For Sale Current'].iloc[latest] + 
                                    balance['Prepaid Assets'].iloc[latest])
    metrics['Current Liabilities'] = balance['Current Liabilities'].iloc[latest] if 'Current Liabilities' in balance.columns else 0
    metrics['Working Capital'] = balance['Working Capital'].iloc[latest] if 'Working Capital' in balance.columns else 0
    if metrics['Working Capital'] == 0 and all(col in balance.columns for col in ['Current Assets', 'Current Liabilities']):
        metrics['Working Capital'] = (metrics['Current Assets'] - metrics['Current Liabilities']) if metrics['Current Assets'] != 0 and metrics['Current Liabilities'] != 0 else 0
    metrics['Operating Cash Flow'] = metrics['CFO']  # Already calculated, stored raw
    metrics['Capital Expenditure'] = abs(cashflow['Capital Expenditure'].iloc[latest]) if 'Capital Expenditure' in cashflow.columns else 0
    metrics['Dividends Paid'] = dividends_paid.iloc[latest] if 'Cash Dividends Paid' in cashflow.columns else 0
    metrics['Depreciation & Amortization'] = dep.iloc[latest] if 'Depreciation And Amortization' in cashflow.columns else 0
    metrics['Interest Expense'] = interest_exp if 'Interest Expense' in income.columns else 0
    metrics['Income Tax Expense'] = income['Income Tax Expense'].iloc[latest] if 'Income Tax Expense' in income.columns else income['Tax Provision'].iloc[latest] if 'Tax Provision' in income.columns else 0
    metrics['Shares Outstanding'] = shares_out
    metrics['Current Price'] = info.get('regularMarketPrice', 0)  # Already calculated, stored raw
    metrics['Net Fixed Assets'] = net_fixed_assets.iloc[latest] if 'Net PPE' in balance.columns else 0
    metrics['Construction in Progress'] = cwip if 'Construction In Progress' in balance.columns else 0
    
    #print(metrics['Total Debt'])
    #print(metrics['Cash and Cash Equivalents'])
    metrics['Net Debt'] = metrics['Total Debt'] - metrics['Cash and Cash Equivalents'] if metrics['Total Debt'] != 0 and metrics['Cash and Cash Equivalents'] != 0 else 0
    
    # ROA and ROE Calculations (Average over 3-5 years)
    net_income_series = income.get('Net Income', pd.Series([0]*num_years))[-min(5, num_years):]
    total_assets_series = balance.get('Total Assets', pd.Series([0]*num_years))[-min(5, num_years):]
    equity_series = balance.get('Stockholders Equity', pd.Series([0]*num_years))[-min(5, num_years):]

    # Calculate yearly ROA and ROE
    years = min(4, num_years)
    roa_values = []
    roe_values = []
    for i in range(years):
        net_income = net_income_series.iloc[i] if not pd.isna(net_income_series.iloc[i]) else 0
        # Average Total Assets for ROA (current + previous) / 2
        if i > 0:
            avg_assets = (total_assets_series.iloc[i] + total_assets_series.iloc[i-1]) / 2
        else:
            avg_assets = total_assets_series.iloc[i]  # Fallback to current if no previous
        if avg_assets != 0:
            roa_values.append((net_income / avg_assets) * 100)
        
        # Average Equity for ROE (current + previous) / 2
        if i > 0:
            avg_equity = (equity_series.iloc[i] + equity_series.iloc[i-1]) / 2
        else:
            avg_equity = equity_series.iloc[i]  # Fallback to current if no previous
        if avg_equity != 0:
            roe_values.append((net_income / avg_equity) * 100)

    roa_values = [x for x in roa_values if pd.notnull(x)]
    roe_values = [x for x in roe_values if pd.notnull(x)]
    # Average ROA and ROE
    metrics['3-5yr Average ROA (%)'] = get_means(roa_values)#np.mean(roa_values) if roa_values else 0
    metrics['3-5yr Average ROE (%)'] = get_means(roe_values)#np.mean(roe_values) if roe_values else 0
    
    metrics['ROE'] = roe_values[latest]
    metrics['ROA'] = roa_values[latest]

    major_holders_dict = dict(stock.major_holders)['Value']
    price_targets = stock.analyst_price_targets
    metrics = {**metrics, **major_holders_dict, **price_targets}

    # Additions based on Damodaran's concepts

    # Normalized Earnings (3-year average as proxy for normalization)
    metrics['Normalized EBIT'] = get_means(ebit[-min(3, len(ebit)):]) #np.mean(ebit[-min(3, num_years):]) if num_years > 0 else 0
    metrics['Normalized Net Income'] = get_means(net_income_series[-min(3, len(net_income_series)):])# np.mean(net_income[-min(3, num_years):]) if num_years > 0 else 0

    # R&D Capitalization (if R&D data available)
    if 'Research And Development' in income.columns:
        rd = income['Research And Development']
        life = 5  # Assumed amortizable life (e.g., 5 years for tech)
        research_asset = 0
        amortization = 0
        current_idx = num_years - 1
        for age in range(0, life):
            idx = current_idx - age
            if idx >= 0:
                rd_value = rd.iloc[idx] if not pd.isna(rd.iloc[idx]) else 0
                unamort_fraction = (life - age) / life
                research_asset += rd_value * unamort_fraction
                amortization += rd_value / life
        metrics['Research Asset'] = research_asset
        metrics['R&D Amortization'] = amortization
        adjusted_ebit = ebit.iloc[latest] + rd.iloc[latest] - amortization if not pd.isna(rd.iloc[latest]) else ebit.iloc[latest]
        metrics['Adjusted EBIT (R&D)'] = adjusted_ebit
        adjusted_book_equity = equity.iloc[latest] + research_asset
        metrics['Adjusted Book Equity (R&D)'] = adjusted_book_equity
        adjusted_de = total_debt / adjusted_book_equity if adjusted_book_equity != 0 else 0
        metrics['Adjusted D/E (R&D)'] = adjusted_de
        tax_rate = metrics['tax %'] / 100 if 'tax %' in metrics and metrics['tax %'] != 0 else 0.25  # Fallback to 25% if missing
        nopat_adjusted = adjusted_ebit * (1 - tax_rate)
        invested_capital_adjusted = adjusted_book_equity + total_debt - metrics.get('Cash and Cash Equivalents', 0)
        metrics['Adjusted Invested Capital (R&D)'] = invested_capital_adjusted
        metrics['Adjusted ROC (R&D)'] = (nopat_adjusted / invested_capital_adjusted * 100) if invested_capital_adjusted != 0 else 0
        adjusted_roe = (net_income_series.iloc[latest] / adjusted_book_equity * 100) if adjusted_book_equity != 0 else 0
        metrics['Adjusted ROE (R&D)'] = adjusted_roe

    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])
    metrics_df.to_csv(f'yfin/{metrics['ticker']}.csv')
    return metrics


def complete_data_fmp(ticker):
    api_key = "XvrsN9KJGVVWKvmIpZ23BubrnrkozwYq"
    #print(f'getting data for {ticker} from financial modelling prep')
    #ticker = "AAPL"  # Example Indian stock
    data = get_peaceful_investing_data_fmp(ticker, api_key)
    print(f'got data for {ticker} from financial modelling prep')
    income, balance, cashflow, key_metrics, profile, quote = data
    metrics = compute_metrics_fmp(income, balance, cashflow, key_metrics, profile, quote)
    print(f'computed metrics for {ticker} from financial modelling prep')
    return metrics

def complete_yfinance_data(ticker):
    #ticker = "GHCL.NS"  # Indian stock
    stock = retrieve_data_from_yfinance(ticker)
    print(f'got data for {ticker} from yfinance')
    metrics=get_peaceful_investing_metrics_from_yfinance(stock)
    print(f'computed metrics for {ticker} from yfinance')
    return metrics


def get_any_ticker_data(ticker):
    try:
        metrics = complete_yfinance_data(ticker)
    except:
        try: 
            metrics =complete_data_fmp(ticker)
        except:pass
    return metrics

def do_everything_for_file_of_stocks(ticker_location, ticker_column, extra_columns, final_file):
    a1 = pd.read_excel(ticker_location)
    list_of_stocks = list(a1[ticker_column])
    new_df = do_everything_for_list_of_stocks(list_of_stocks)
    fin_df = a1[[ticker_column ]+extra_columns ].merge(new_df, right_on ='ticker', left_on = ticker_column, how =  'left' )
    #finviz_list_1_cheap_us_growing
    fin_df.to_csv(f'combined_stocks/{final_file}.csv')


def do_everything_for_list_of_stocks(list_of_stocks):
    for st in list_of_stocks:
        try:
            metrics = complete_yfinance_data(st)
        except:
            print(f'couldnt do : {st}')
        time.sleep(1)
        print(f'done ticker : {st}')

    list_dfs = []
    for st in list_of_stocks:
        try:
            p1 =pd.read_csv(f'yfin/{st}.csv')
            columns_order = p1['Unnamed: 0'].values
            p1= p1.fillna(-1)
            piv_df = p1.pivot_table(index=None, columns='Unnamed: 0', values='Value', aggfunc='first').reset_index(drop=True)[columns_order]
            list_dfs.append(piv_df)
        except:print(f"not done : {st}")
    new_df = pd.concat(list_dfs)
    return new_df

In [37]:
ticker = yf.Ticker("AAPL")
df = ticker.history(period="6mo") 

In [39]:
# Calculate EMAs
df['EMA12'] = df['Close'].ewm(span=12).mean()
df['EMA26'] = df['Close'].ewm(span=26).mean()

In [1]:
from agents.technical_analysis_agent.technical_analysis import complete_technical_analysis

ModuleNotFoundError: No module named 'agents'

In [ ]:
a3 = pd.read_csv('raw_tickers/RAHUL_RECOS.txt', header = None)
list_stocks =[x.split()[0].split(':')[1] for x in a3.loc[0].values]
df = do_everything_for_list_of_stocks(list_stocks)   
df.to_csv('combined_stocks/rahul_recos.csv')

In [507]:
a3 = pd.read_excel('raw_tickers/indian_stocks_15_30_pe.xlsx')
#a1 = pd.read_excel('raw_tickers/finviz_list_1_us_cheap_growing.xlsx')
list_of_stocks = list(a3['stock'])
do_everything_for_list_of_stocks(list_of_stocks, 'indian_stocks_15_30_pe')  

got data for 5PAISA.NS from yfinance
computed metrics for 5PAISA.NS from yfinance
done ticker : 5PAISA.NS
got data for AADHARHFC.NS from yfinance
computed metrics for AADHARHFC.NS from yfinance
done ticker : AADHARHFC.NS
got data for AARTIPHARM.NS from yfinance
computed metrics for AARTIPHARM.NS from yfinance
done ticker : AARTIPHARM.NS
got data for AAVAS.NS from yfinance
computed metrics for AAVAS.NS from yfinance
done ticker : AAVAS.NS
got data for ABSLAMC.NS from yfinance
computed metrics for ABSLAMC.NS from yfinance
done ticker : ABSLAMC.NS
got data for ACC.NS from yfinance
computed metrics for ACC.NS from yfinance
done ticker : ACC.NS
got data for ACCELYA.NS from yfinance
computed metrics for ACCELYA.NS from yfinance
done ticker : ACCELYA.NS
got data for ADANIPORTS.NS from yfinance
computed metrics for ADANIPORTS.NS from yfinance
done ticker : ADANIPORTS.NS
got data for ADANIPOWER.NS from yfinance
computed metrics for ADANIPOWER.NS from yfinance
done ticker : ADANIPOWER.NS
got dat

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


couldnt do : CROWN.NS
done ticker : CROWN.NS
got data for CYBERTECH.NS from yfinance
computed metrics for CYBERTECH.NS from yfinance
done ticker : CYBERTECH.NS
got data for DATAMATICS.NS from yfinance
computed metrics for DATAMATICS.NS from yfinance
done ticker : DATAMATICS.NS
got data for DBL.NS from yfinance
computed metrics for DBL.NS from yfinance
done ticker : DBL.NS
got data for DBSTOCKBRO.NS from yfinance
computed metrics for DBSTOCKBRO.NS from yfinance
done ticker : DBSTOCKBRO.NS
got data for DEEPINDS.NS from yfinance
computed metrics for DEEPINDS.NS from yfinance
done ticker : DEEPINDS.NS
got data for DGCONTENT.NS from yfinance
computed metrics for DGCONTENT.NS from yfinance
done ticker : DGCONTENT.NS
got data for DHANUKA.NS from yfinance
computed metrics for DHANUKA.NS from yfinance
done ticker : DHANUKA.NS
got data for DLINKINDIA.NS from yfinance
computed metrics for DLINKINDIA.NS from yfinance
done ticker : DLINKINDIA.NS
got data for DRREDDY.NS from yfinance
computed metric

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


couldnt do : PAR.NS
done ticker : PAR.NS
got data for PARACABLES.NS from yfinance
computed metrics for PARACABLES.NS from yfinance
done ticker : PARACABLES.NS
got data for PARAGMILK.NS from yfinance
computed metrics for PARAGMILK.NS from yfinance
done ticker : PARAGMILK.NS
got data for PATINTLOG.NS from yfinance
computed metrics for PATINTLOG.NS from yfinance
done ticker : PATINTLOG.NS
got data for PCBL.NS from yfinance
computed metrics for PCBL.NS from yfinance
done ticker : PCBL.NS
got data for PFS.NS from yfinance
computed metrics for PFS.NS from yfinance
done ticker : PFS.NS
got data for PLASTIBLEN.NS from yfinance
computed metrics for PLASTIBLEN.NS from yfinance
done ticker : PLASTIBLEN.NS
got data for PREMIERPOL.NS from yfinance
computed metrics for PREMIERPOL.NS from yfinance
done ticker : PREMIERPOL.NS
got data for PRIMESECU.NS from yfinance
computed metrics for PRIMESECU.NS from yfinance
done ticker : PRIMESECU.NS
got data for PSPPROJECT.NS from yfinance
computed metrics for P

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for SELAN.NS from yfinance
done ticker : SELAN.NS
got data for SHAKTIPUMP.NS from yfinance
computed metrics for SHAKTIPUMP.NS from yfinance
done ticker : SHAKTIPUMP.NS
got data for SHANKARA.NS from yfinance
couldnt do : SHANKARA.NS
done ticker : SHANKARA.NS
got data for SHRIPISTON.NS from yfinance
computed metrics for SHRIPISTON.NS from yfinance
done ticker : SHRIPISTON.NS
got data for SHYAMMETL.NS from yfinance
computed metrics for SHYAMMETL.NS from yfinance
done ticker : SHYAMMETL.NS
got data for SIKKO.NS from yfinance
couldnt do : SIKKO.NS
done ticker : SIKKO.NS
got data for SMARTLINK.NS from yfinance
computed metrics for SMARTLINK.NS from yfinance
done ticker : SMARTLINK.NS
got data for SPANDANA.NS from yfinance
computed metrics for SPANDANA.NS from yfinance
done ticker : SPANDANA.NS
got data for SRGHFL.NS from yfinance
computed metrics for SRGHFL.NS from yfinance
done ticker : SRGHFL.NS
got data for STYLAMIND.NS from yfinance
computed metrics for STYLAMIND.NS from

In [502]:
a2 = pd.read_excel('raw_tickers/indian_stonks.xlsx')
#a1 = pd.read_excel('raw_tickers/finviz_list_1_us_cheap_growing.xlsx')
list_of_stocks = list(a2['stock'])
do_everything_for_list_of_stocks(list_of_stocks, 'indian_stonks')   

got data for WSI.NS from yfinance
computed metrics for WSI.NS from yfinance
done ticker : WSI.NS
got data for VIPULLTD.NS from yfinance
computed metrics for VIPULLTD.NS from yfinance
done ticker : VIPULLTD.NS
got data for VINDHYATEL.NS from yfinance
computed metrics for VINDHYATEL.NS from yfinance
done ticker : VINDHYATEL.NS
got data for UTKARSHBNK.NS from yfinance
computed metrics for UTKARSHBNK.NS from yfinance
done ticker : UTKARSHBNK.NS
got data for UNITEDTEA.NS from yfinance
couldnt do : UNITEDTEA.NS
done ticker : UNITEDTEA.NS
got data for UNIONBANK.NS from yfinance
computed metrics for UNIONBANK.NS from yfinance
done ticker : UNIONBANK.NS
got data for UJJIVANSFB.NS from yfinance
computed metrics for UJJIVANSFB.NS from yfinance
done ticker : UJJIVANSFB.NS
got data for TVTODAY.NS from yfinance
computed metrics for TVTODAY.NS from yfinance
done ticker : TVTODAY.NS
got data for TMB.NS from yfinance
computed metrics for TMB.NS from yfinance
done ticker : TMB.NS
got data for TIL.NS fro

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for KIRIINDUS.NS from yfinance
done ticker : KIRIINDUS.NS
got data for KHAITANLTD.NS from yfinance
computed metrics for KHAITANLTD.NS from yfinance
done ticker : KHAITANLTD.NS
got data for KCP.NS from yfinance
computed metrics for KCP.NS from yfinance
done ticker : KCP.NS
got data for KARURVYSYA.NS from yfinance
computed metrics for KARURVYSYA.NS from yfinance
done ticker : KARURVYSYA.NS
got data for KAMATHOTEL.NS from yfinance
computed metrics for KAMATHOTEL.NS from yfinance
done ticker : KAMATHOTEL.NS
got data for JSFB.NS from yfinance
computed metrics for JSFB.NS from yfinance
done ticker : JSFB.NS
got data for JPPOWER.NS from yfinance
computed metrics for JPPOWER.NS from yfinance
done ticker : JPPOWER.NS
got data for JKPAPER.NS from yfinance
computed metrics for JKPAPER.NS from yfinance
done ticker : JKPAPER.NS
got data for JKIL.NS from yfinance
computed metrics for JKIL.NS from yfinance
done ticker : JKIL.NS
got data for JINDALSAW.NS from yfinance
computed metrics

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for FIBERWEB.NS from yfinance
done ticker : FIBERWEB.NS
got data for FEDFINA.NS from yfinance
computed metrics for FEDFINA.NS from yfinance
done ticker : FEDFINA.NS
got data for FEDERALBNK.NS from yfinance
computed metrics for FEDERALBNK.NS from yfinance
done ticker : FEDERALBNK.NS
got data for EMKAY.NS from yfinance
computed metrics for EMKAY.NS from yfinance
done ticker : EMKAY.NS
got data for ELECTCAST.NS from yfinance
computed metrics for ELECTCAST.NS from yfinance
done ticker : ELECTCAST.NS
got data for DCBBANK.NS from yfinance
computed metrics for DCBBANK.NS from yfinance
done ticker : DCBBANK.NS
got data for DBCORP.NS from yfinance
computed metrics for DBCORP.NS from yfinance
done ticker : DBCORP.NS
got data for CUB.NS from yfinance
computed metrics for CUB.NS from yfinance
done ticker : CUB.NS
got data for CSBBANK.NS from yfinance
couldnt do : CSBBANK.NS
done ticker : CSBBANK.NS
got data for CREDITACC.NS from yfinance
computed metrics for CREDITACC.NS from yfin

In [ ]:
a1 = pd.read_excel('raw_tickers/finviz_list_1_us_cheap_growing.xlsx')
list_of_stocks = list(a1['Ticker'])
do_everything_for_list_of_stocks(list_of_stocks, 'finviz_list_1_cheap_us_growing')

In [35]:
do_everything_for_file_of_stocks('raw_tickers/indian_stonks.xlsx', 'stock', ['name','sector','industry'], 'indian_stonks')

got data for WSI.NS from yfinance
computed metrics for WSI.NS from yfinance
done ticker : WSI.NS
got data for VIPULLTD.NS from yfinance
computed metrics for VIPULLTD.NS from yfinance
done ticker : VIPULLTD.NS
got data for VINDHYATEL.NS from yfinance
computed metrics for VINDHYATEL.NS from yfinance
done ticker : VINDHYATEL.NS
got data for UTKARSHBNK.NS from yfinance
computed metrics for UTKARSHBNK.NS from yfinance
done ticker : UTKARSHBNK.NS
got data for UNITEDTEA.NS from yfinance
couldnt do : UNITEDTEA.NS
done ticker : UNITEDTEA.NS
got data for UNIONBANK.NS from yfinance
computed metrics for UNIONBANK.NS from yfinance
done ticker : UNIONBANK.NS
got data for UJJIVANSFB.NS from yfinance
computed metrics for UJJIVANSFB.NS from yfinance
done ticker : UJJIVANSFB.NS
got data for TVTODAY.NS from yfinance
computed metrics for TVTODAY.NS from yfinance
done ticker : TVTODAY.NS
got data for TMB.NS from yfinance
computed metrics for TMB.NS from yfinance
done ticker : TMB.NS
got data for TIL.NS fro

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for KIRIINDUS.NS from yfinance
done ticker : KIRIINDUS.NS
got data for KHAITANLTD.NS from yfinance
computed metrics for KHAITANLTD.NS from yfinance
done ticker : KHAITANLTD.NS
got data for KCP.NS from yfinance
computed metrics for KCP.NS from yfinance
done ticker : KCP.NS
got data for KARURVYSYA.NS from yfinance
computed metrics for KARURVYSYA.NS from yfinance
done ticker : KARURVYSYA.NS
got data for KAMATHOTEL.NS from yfinance
computed metrics for KAMATHOTEL.NS from yfinance
done ticker : KAMATHOTEL.NS
got data for JSFB.NS from yfinance
computed metrics for JSFB.NS from yfinance
done ticker : JSFB.NS
got data for JPPOWER.NS from yfinance
computed metrics for JPPOWER.NS from yfinance
done ticker : JPPOWER.NS
got data for JKPAPER.NS from yfinance
computed metrics for JKPAPER.NS from yfinance
done ticker : JKPAPER.NS
got data for JKIL.NS from yfinance
computed metrics for JKIL.NS from yfinance
done ticker : JKIL.NS
got data for JINDALSAW.NS from yfinance
computed metrics

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for FIBERWEB.NS from yfinance
done ticker : FIBERWEB.NS
got data for FEDFINA.NS from yfinance
computed metrics for FEDFINA.NS from yfinance
done ticker : FEDFINA.NS
got data for FEDERALBNK.NS from yfinance
computed metrics for FEDERALBNK.NS from yfinance
done ticker : FEDERALBNK.NS
got data for EMKAY.NS from yfinance
computed metrics for EMKAY.NS from yfinance
done ticker : EMKAY.NS
got data for ELECTCAST.NS from yfinance
computed metrics for ELECTCAST.NS from yfinance
done ticker : ELECTCAST.NS
got data for DCBBANK.NS from yfinance
computed metrics for DCBBANK.NS from yfinance
done ticker : DCBBANK.NS
got data for DBCORP.NS from yfinance
computed metrics for DBCORP.NS from yfinance
done ticker : DBCORP.NS
got data for CUB.NS from yfinance
computed metrics for CUB.NS from yfinance
done ticker : CUB.NS
got data for CSBBANK.NS from yfinance
couldnt do : CSBBANK.NS
done ticker : CSBBANK.NS
got data for CREDITACC.NS from yfinance
computed metrics for CREDITACC.NS from yfin

In [36]:
do_everything_for_file_of_stocks('raw_tickers/indian_stocks_15_30_pe.xlsx', 'stock', ['name','sector','industry'], 'indian_stocks_15_30_pe')

got data for 5PAISA.NS from yfinance
computed metrics for 5PAISA.NS from yfinance
done ticker : 5PAISA.NS
got data for AADHARHFC.NS from yfinance
computed metrics for AADHARHFC.NS from yfinance
done ticker : AADHARHFC.NS
got data for AARTIPHARM.NS from yfinance
computed metrics for AARTIPHARM.NS from yfinance
done ticker : AARTIPHARM.NS
got data for AAVAS.NS from yfinance
computed metrics for AAVAS.NS from yfinance
done ticker : AAVAS.NS
got data for ABSLAMC.NS from yfinance
computed metrics for ABSLAMC.NS from yfinance
done ticker : ABSLAMC.NS
got data for ACC.NS from yfinance
computed metrics for ACC.NS from yfinance
done ticker : ACC.NS
got data for ACCELYA.NS from yfinance
computed metrics for ACCELYA.NS from yfinance
done ticker : ACCELYA.NS
got data for ADANIPORTS.NS from yfinance
computed metrics for ADANIPORTS.NS from yfinance
done ticker : ADANIPORTS.NS
got data for ADANIPOWER.NS from yfinance
computed metrics for ADANIPOWER.NS from yfinance
done ticker : ADANIPOWER.NS
got dat

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


couldnt do : CROWN.NS
done ticker : CROWN.NS
got data for CYBERTECH.NS from yfinance
computed metrics for CYBERTECH.NS from yfinance
done ticker : CYBERTECH.NS
got data for DATAMATICS.NS from yfinance
computed metrics for DATAMATICS.NS from yfinance
done ticker : DATAMATICS.NS
got data for DBL.NS from yfinance
computed metrics for DBL.NS from yfinance
done ticker : DBL.NS
got data for DBSTOCKBRO.NS from yfinance
computed metrics for DBSTOCKBRO.NS from yfinance
done ticker : DBSTOCKBRO.NS
got data for DEEPINDS.NS from yfinance
computed metrics for DEEPINDS.NS from yfinance
done ticker : DEEPINDS.NS
got data for DGCONTENT.NS from yfinance
computed metrics for DGCONTENT.NS from yfinance
done ticker : DGCONTENT.NS
got data for DHANUKA.NS from yfinance
computed metrics for DHANUKA.NS from yfinance
done ticker : DHANUKA.NS
got data for DLINKINDIA.NS from yfinance
computed metrics for DLINKINDIA.NS from yfinance
done ticker : DLINKINDIA.NS
got data for DRREDDY.NS from yfinance
computed metric

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


couldnt do : PAR.NS
done ticker : PAR.NS
got data for PARACABLES.NS from yfinance
computed metrics for PARACABLES.NS from yfinance
done ticker : PARACABLES.NS
got data for PARAGMILK.NS from yfinance
computed metrics for PARAGMILK.NS from yfinance
done ticker : PARAGMILK.NS
got data for PATINTLOG.NS from yfinance
computed metrics for PATINTLOG.NS from yfinance
done ticker : PATINTLOG.NS
got data for PCBL.NS from yfinance
computed metrics for PCBL.NS from yfinance
done ticker : PCBL.NS
got data for PFS.NS from yfinance
computed metrics for PFS.NS from yfinance
done ticker : PFS.NS
got data for PLASTIBLEN.NS from yfinance
computed metrics for PLASTIBLEN.NS from yfinance
done ticker : PLASTIBLEN.NS
got data for PREMIERPOL.NS from yfinance
computed metrics for PREMIERPOL.NS from yfinance
done ticker : PREMIERPOL.NS
got data for PRIMESECU.NS from yfinance
computed metrics for PRIMESECU.NS from yfinance
done ticker : PRIMESECU.NS
got data for PSPPROJECT.NS from yfinance
computed metrics for P

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for SELAN.NS from yfinance
done ticker : SELAN.NS
got data for SHAKTIPUMP.NS from yfinance
computed metrics for SHAKTIPUMP.NS from yfinance
done ticker : SHAKTIPUMP.NS
got data for SHANKARA.NS from yfinance
couldnt do : SHANKARA.NS
done ticker : SHANKARA.NS
got data for SHRIPISTON.NS from yfinance
computed metrics for SHRIPISTON.NS from yfinance
done ticker : SHRIPISTON.NS
got data for SHYAMMETL.NS from yfinance
computed metrics for SHYAMMETL.NS from yfinance
done ticker : SHYAMMETL.NS
got data for SIKKO.NS from yfinance
couldnt do : SIKKO.NS
done ticker : SIKKO.NS
got data for SMARTLINK.NS from yfinance
computed metrics for SMARTLINK.NS from yfinance
done ticker : SMARTLINK.NS
got data for SPANDANA.NS from yfinance
computed metrics for SPANDANA.NS from yfinance
done ticker : SPANDANA.NS
got data for SRGHFL.NS from yfinance
computed metrics for SRGHFL.NS from yfinance
done ticker : SRGHFL.NS
got data for STYLAMIND.NS from yfinance
computed metrics for STYLAMIND.NS from

In [49]:
do_everything_for_file_of_stocks('raw_tickers/us_growth_20.xlsx', 'Ticker', ['Company','Sector','Industry','P/E'], 'us_growth_20')

got data for AAON from yfinance
computed metrics for AAON from yfinance
done ticker : AAON
got data for ABR from yfinance
computed metrics for ABR from yfinance
done ticker : ABR
got data for ACAD from yfinance
computed metrics for ACAD from yfinance
done ticker : ACAD
got data for ACLS from yfinance
computed metrics for ACLS from yfinance
done ticker : ACLS
got data for ACMR from yfinance
computed metrics for ACMR from yfinance
done ticker : ACMR
got data for ADC from yfinance
computed metrics for ADC from yfinance
done ticker : ADC
got data for ADMA from yfinance
computed metrics for ADMA from yfinance
done ticker : ADMA
got data for AFBI from yfinance
computed metrics for AFBI from yfinance
done ticker : AFBI
got data for AGNC from yfinance
computed metrics for AGNC from yfinance
done ticker : AGNC
got data for AGX from yfinance
computed metrics for AGX from yfinance
done ticker : AGX
got data for AMD from yfinance
computed metrics for AMD from yfinance
done ticker : AMD
got data fo

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for CROX from yfinance
done ticker : CROX
got data for CTRA from yfinance
computed metrics for CTRA from yfinance
done ticker : CTRA
got data for CUBI from yfinance
computed metrics for CUBI from yfinance
done ticker : CUBI
got data for CWAN from yfinance
computed metrics for CWAN from yfinance
done ticker : CWAN
got data for DASH from yfinance
computed metrics for DASH from yfinance
done ticker : DASH
got data for DCOM from yfinance
computed metrics for DCOM from yfinance
done ticker : DCOM
got data for DFH from yfinance
computed metrics for DFH from yfinance
done ticker : DFH
got data for DLO from yfinance
computed metrics for DLO from yfinance
done ticker : DLO
got data for DOCN from yfinance
computed metrics for DOCN from yfinance
done ticker : DOCN
got data for DOCS from yfinance
computed metrics for DOCS from yfinance
done ticker : DOCS
got data for DOCU from yfinance
computed metrics for DOCU from yfinance
done ticker : DOCU
got data for DT from yfinance
compute

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for FTLF from yfinance
done ticker : FTLF
got data for FTNT from yfinance
computed metrics for FTNT from yfinance
done ticker : FTNT
got data for FUBO from yfinance
computed metrics for FUBO from yfinance
done ticker : FUBO
got data for GBFH from yfinance
computed metrics for GBFH from yfinance
done ticker : GBFH
got data for GCT from yfinance
computed metrics for GCT from yfinance
done ticker : GCT
got data for GMED from yfinance
computed metrics for GMED from yfinance
done ticker : GMED
got data for GRBK from yfinance
computed metrics for GRBK from yfinance
done ticker : GRBK
got data for GSHD from yfinance
computed metrics for GSHD from yfinance
done ticker : GSHD
got data for GTLS from yfinance
computed metrics for GTLS from yfinance
done ticker : GTLS
got data for GTM from yfinance
computed metrics for GTM from yfinance
done ticker : GTM
got data for HALO from yfinance
computed metrics for HALO from yfinance
done ticker : HALO
got data for HASI from yfinance
compu

...

computed metrics for SHLS from yfinance
done ticker : SHLS
got data for SIGA from yfinance
computed metrics for SIGA from yfinance
done ticker : SIGA
got data for SKWD from yfinance
computed metrics for SKWD from yfinance
done ticker : SKWD
got data for SLSN from yfinance
computed metrics for SLSN from yfinance
done ticker : SLSN
got data for SMBC from yfinance
computed metrics for SMBC from yfinance
done ticker : SMBC
got data for SMPL from yfinance
computed metrics for SMPL from yfinance
done ticker : SMPL
got data for SOFI from yfinance
computed metrics for SOFI from yfinance
done ticker : SOFI
got data for SSB from yfinance
computed metrics for SSB from yfinance
done ticker : SSB
got data for STEL from yfinance
computed metrics for STEL from yfinance
done ticker : STEL
got data for STR from yfinance
couldnt do : STR
done ticker : STR
got data for SUI from yfinance
computed metrics for SUI from yfinance
done ticker : SUI
got data for SYBT from yfinance
computed metrics for SYBT from

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for TGTX from yfinance
done ticker : TGTX
got data for TKO from yfinance
couldnt do : TKO
done ticker : TKO
got data for TMDX from yfinance
computed metrics for TMDX from yfinance
done ticker : TMDX
got data for TSLA from yfinance
computed metrics for TSLA from yfinance
done ticker : TSLA
got data for TTC from yfinance
computed metrics for TTC from yfinance
done ticker : TTC
got data for TTD from yfinance
computed metrics for TTD from yfinance
done ticker : TTD
got data for UBER from yfinance
computed metrics for UBER from yfinance
done ticker : UBER
got data for UFPT from yfinance
computed metrics for UFPT from yfinance
done ticker : UFPT
got data for UPWK from yfinance
computed metrics for UPWK from yfinance
done ticker : UPWK
got data for USCB from yfinance
computed metrics for USCB from yfinance
done ticker : USCB
got data for VABK from yfinance
computed metrics for VABK from yfinance
done ticker : VABK
got data for VCYT from yfinance
computed metrics for VCYT from

/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_69181/2744554039.py:527: RuntimeWarning: divide by zero encountered in scalar divide
  de_ratios = [balance.get('Total Debt', pd.Series([0]*num_years)).iloc[i] / balance.get('Stockholders Equity', pd.Series([1]*num_years)).iloc[i] for i in range(-min(5, num_years), 0)]


computed metrics for VNOM from yfinance
done ticker : VNOM
got data for VRTX from yfinance
couldnt do : VRTX
done ticker : VRTX
got data for VTOL from yfinance
computed metrics for VTOL from yfinance
done ticker : VTOL
got data for WAL from yfinance
computed metrics for WAL from yfinance
done ticker : WAL
got data for WBS from yfinance
computed metrics for WBS from yfinance
done ticker : WBS
got data for WING from yfinance
computed metrics for WING from yfinance
done ticker : WING
got data for WTM from yfinance
computed metrics for WTM from yfinance
done ticker : WTM
got data for XPEL from yfinance
computed metrics for XPEL from yfinance
done ticker : XPEL
got data for XYZ from yfinance
computed metrics for XYZ from yfinance
done ticker : XYZ
got data for YOU from yfinance
computed metrics for YOU from yfinance
done ticker : YOU
got data for ZM from yfinance
computed metrics for ZM from yfinance
done ticker : ZM
not done : AVBH
not done : COFS
not done : DXCM
not done : ECBK
not done :

In [37]:
a1 = pd.read_csv('combined_stocks/indian_stocks_15_30_pe.csv')
a2 = pd.read_csv('combined_stocks/indian_stonks.csv')
a3 = pd.concat([a1,a2])

In [48]:
pd.qcut(a3['3-5yr Average ROE (%)'], [0, 0.25, 0.5, 0.75, 1]).value_counts()

3-5yr Average ROE (%)
(-562.456, 9.158]     82
(18.886, 2496.457]    82
(9.158, 13.889]       81
(13.889, 18.886]      81
Name: count, dtype: int64

In [39]:
a3.to_csv('combined_stocks/india_0_30.csv')

In [96]:

do_everything_for_file_of_stocks('raw_tickers/hedge_fund_buys.xlsx', 'symbol', ['name'], 'hedge_fund_buys')

got data for AAPL from yfinance
computed metrics for AAPL from yfinance
done ticker : AAPL
got data for AMZN from yfinance
computed metrics for AMZN from yfinance
done ticker : AMZN
got data for NVDA from yfinance
computed metrics for NVDA from yfinance
done ticker : NVDA
got data for MSFT from yfinance
computed metrics for MSFT from yfinance
done ticker : MSFT
got data for COF from yfinance
computed metrics for COF from yfinance
done ticker : COF
got data for UNH from yfinance
computed metrics for UNH from yfinance
done ticker : UNH
got data for SPY from yfinance
computed metrics for SPY from yfinance
done ticker : SPY
got data for AVGO from yfinance
computed metrics for AVGO from yfinance
done ticker : AVGO
got data for META from yfinance
computed metrics for META from yfinance
done ticker : META
got data for GOOGL from yfinance
computed metrics for GOOGL from yfinance
done ticker : GOOGL
got data for TSLA from yfinance
computed metrics for TSLA from yfinance
done ticker : TSLA
got d

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for CVNA from yfinance
done ticker : CVNA
got data for APP from yfinance
computed metrics for APP from yfinance
done ticker : APP
got data for SPOT from yfinance
computed metrics for SPOT from yfinance
done ticker : SPOT
got data for MRK from yfinance
computed metrics for MRK from yfinance
done ticker : MRK
got data for UNP from yfinance
computed metrics for UNP from yfinance
done ticker : UNP
got data for PEP from yfinance
computed metrics for PEP from yfinance
done ticker : PEP
got data for BMY from yfinance
computed metrics for BMY from yfinance
done ticker : BMY
got data for AMCR from yfinance
computed metrics for AMCR from yfinance
done ticker : AMCR
got data for XYZ from yfinance
computed metrics for XYZ from yfinance
done ticker : XYZ
got data for REGN from yfinance
computed metrics for REGN from yfinance
done ticker : REGN
got data for ADBE from yfinance
computed metrics for ADBE from yfinance
done ticker : ADBE
got data for APH from yfinance
computed metrics f

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for BA from yfinance
done ticker : BA
got data for AON from yfinance
computed metrics for AON from yfinance
done ticker : AON
got data for LIN from yfinance
computed metrics for LIN from yfinance
done ticker : LIN
got data for RDDT from yfinance
computed metrics for RDDT from yfinance
done ticker : RDDT
got data for VRTX from yfinance
computed metrics for VRTX from yfinance
done ticker : VRTX
got data for ISRG from yfinance
computed metrics for ISRG from yfinance
done ticker : ISRG
got data for VST from yfinance
computed metrics for VST from yfinance
done ticker : VST
got data for MU from yfinance
computed metrics for MU from yfinance
done ticker : MU
not done : SPY
not done : BRKB
not done : QQQ
not done : GBIL
not done : IBIT
not done : GEV
not done : IVV
not done : VOO


In [113]:
mf = pd.read_html('raw_tickers/magic_formula.xml')[0].rename(columns = {0:'name',1:'symbol'})
mf.to_excel('raw_tickers/magic_formula.xlsx')

In [115]:
do_everything_for_file_of_stocks('raw_tickers/magic_formula.xlsx', 'symbol', ['name'], 'magic_formula')

got data for ANF from yfinance
computed metrics for ANF from yfinance
done ticker : ANF
got data for MO from yfinance
computed metrics for MO from yfinance
done ticker : MO
got data for AMCX from yfinance
computed metrics for AMCX from yfinance
done ticker : AMCX
got data for ATRA from yfinance
computed metrics for ATRA from yfinance
done ticker : ATRA
got data for BTMD from yfinance
computed metrics for BTMD from yfinance
done ticker : BTMD
got data for BBUC from yfinance
computed metrics for BBUC from yfinance
done ticker : BBUC
got data for BMBL from yfinance
computed metrics for BMBL from yfinance
done ticker : BMBL
got data for CPRX from yfinance
computed metrics for CPRX from yfinance
done ticker : CPRX
got data for CCSI from yfinance
computed metrics for CCSI from yfinance
done ticker : CCSI
got data for CROX from yfinance


/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for CROX from yfinance
done ticker : CROX
got data for CTMX from yfinance
computed metrics for CTMX from yfinance
done ticker : CTMX
got data for FOXA from yfinance
computed metrics for FOXA from yfinance
done ticker : FOXA
got data for FC from yfinance
computed metrics for FC from yfinance
done ticker : FC
got data for GAMB from yfinance
computed metrics for GAMB from yfinance
done ticker : GAMB
got data for GETY from yfinance
computed metrics for GETY from yfinance
done ticker : GETY
got data for GCT from yfinance
computed metrics for GCT from yfinance
done ticker : GCT
got data for HRB from yfinance
computed metrics for HRB from yfinance
done ticker : HRB
got data for HRMY from yfinance
computed metrics for HRMY from yfinance
done ticker : HRMY
got data for HSII from yfinance
computed metrics for HSII from yfinance
done ticker : HSII
got data for HPQ from yfinance
computed metrics for HPQ from yfinance
done ticker : HPQ
got data for INVA from yfinance


/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


computed metrics for INVA from yfinance
done ticker : INVA
got data for IPG from yfinance
computed metrics for IPG from yfinance
done ticker : IPG
got data for IRWD from yfinance
computed metrics for IRWD from yfinance
done ticker : IRWD
got data for JILL from yfinance
computed metrics for JILL from yfinance
done ticker : JILL
got data for KFY from yfinance
computed metrics for KFY from yfinance
done ticker : KFY
got data for LNTH from yfinance
computed metrics for LNTH from yfinance
done ticker : LNTH
got data for NL from yfinance
computed metrics for NL from yfinance
done ticker : NL
got data for OMC from yfinance
computed metrics for OMC from yfinance
done ticker : OMC
got data for OSPN from yfinance
computed metrics for OSPN from yfinance
done ticker : OSPN
got data for OMI from yfinance
computed metrics for OMI from yfinance
done ticker : OMI
got data for MD from yfinance
computed metrics for MD from yfinance
done ticker : MD
got data for PRDO from yfinance
computed metrics for PR

/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_69181/2324962218.py:537: RuntimeWarning: divide by zero encountered in scalar divide
  de_ratios = [balance.get('Total Debt', pd.Series([0]*num_years)).iloc[i] / balance.get('Stockholders Equity', pd.Series([1]*num_years)).iloc[i] for i in range(-min(3, num_years), 0)]


couldnt do : RHLD
done ticker : RHLD
got data for RIGL from yfinance
computed metrics for RIGL from yfinance
done ticker : RIGL
got data for RMNI from yfinance
computed metrics for RMNI from yfinance
done ticker : RMNI
got data for SBC from yfinance
computed metrics for SBC from yfinance
done ticker : SBC
got data for SSTK from yfinance
computed metrics for SSTK from yfinance
done ticker : SSTK
got data for SIGA from yfinance
computed metrics for SIGA from yfinance
done ticker : SIGA
got data for SIRI from yfinance
computed metrics for SIRI from yfinance
done ticker : SIRI
got data for TGNA from yfinance
computed metrics for TGNA from yfinance
done ticker : TGNA
got data for TZOO from yfinance
computed metrics for TZOO from yfinance
done ticker : TZOO
got data for UIS from yfinance
computed metrics for UIS from yfinance
done ticker : UIS
got data for WW from yfinance
computed metrics for WW from yfinance
done ticker : WW
got data for XPOF from yfinance
computed metrics for XPOF from yf

In [117]:
do_everything_for_file_of_stocks('raw_tickers/india_portfolio_tickers.xlsx', 'symbol', [], 'india_portfolio_tickers')

got data for ITCHOTELS.NS from yfinance
couldnt do : ITCHOTELS.NS
done ticker : ITCHOTELS.NS
got data for JIOFIN.NS from yfinance
computed metrics for JIOFIN.NS from yfinance
done ticker : JIOFIN.NS
got data for LT.NS from yfinance
computed metrics for LT.NS from yfinance
done ticker : LT.NS
got data for NETWEB.NS from yfinance
computed metrics for NETWEB.NS from yfinance
done ticker : NETWEB.NS
got data for TITAN.NS from yfinance
computed metrics for TITAN.NS from yfinance
done ticker : TITAN.NS
got data for HDFCAMC.NS from yfinance
computed metrics for HDFCAMC.NS from yfinance
done ticker : HDFCAMC.NS
got data for HDFCLIFE.NS from yfinance
computed metrics for HDFCLIFE.NS from yfinance
done ticker : HDFCLIFE.NS
got data for ICICIGI.NS from yfinance
computed metrics for ICICIGI.NS from yfinance
done ticker : ICICIGI.NS
got data for INDIACEM.NS from yfinance
computed metrics for INDIACEM.NS from yfinance
done ticker : INDIACEM.NS
got data for INFY.NS from yfinance
computed metrics for 

In [55]:
df = do_everything_for_list_of_stocks(['LULU','BRKR','REGN','MELI','UNH'])

got data for LULU from yfinance
computed metrics for LULU from yfinance
done ticker : LULU
got data for BRKR from yfinance
computed metrics for BRKR from yfinance
done ticker : BRKR
got data for REGN from yfinance
computed metrics for REGN from yfinance
done ticker : REGN
got data for MELI from yfinance
computed metrics for MELI from yfinance
done ticker : MELI
got data for UNH from yfinance
computed metrics for UNH from yfinance
done ticker : UNH


In [56]:
df.to_csv('combined_stocks/scion_amc.csv')

In [89]:
stock = retrieve_data_from_yfinance('COST')

In [94]:
metrics=get_peaceful_investing_metrics_from_yfinance(stock)